In [44]:
import numpy as np
import pandas as pd
from setuptools.wheel import unpack
from ydata_profiling import ProfileReport

df = pd.read_csv("./dataset/request.csv")
df['request_method'] = df['customDimensions'].apply(lambda x: eval(x).get('http.request.method'))
df['request_url'] = df['customDimensions'].apply(lambda x: eval(x).get('http.request.url'))
df['request_url'] = df['request_url'].str.replace(r'/chat_with_data/[\w]{32}', '/chat_with_data/{session_id}', regex=True)
df['request_url'] = df['request_url'].str.replace(r'/multimodal/[\w]{32}', '/multimodal/{session_id}', regex=True)
df['request_line'] = df['request_method'] + ' ' + df['request_url']
df['response_code'] = df['customDimensions'].apply(lambda x: eval(x).get('http.response.status_code'))
df['response_time_ms'] = df['customDimensions'].apply(lambda x: eval(x).get('http.response_time_ms'))

# remove rows where response_code and response_time_ms is None
df = df[(df['response_code'].notna()) & (df['response_time_ms'].notna())]

# convert response_code to int, response_time_ms to float
df['response_code'] = df['response_code'].astype(int)
df['response_time_ms'] = df['response_time_ms'].astype(float)

df = df.rename(columns={'timestamp [UTC]': 'timestamp'})
df = df [['timestamp', 'request_line', 'response_code', 'response_time_ms']]
df.head(5)

,timestamp,request_line,response_code,response_time_ms
0,"9/24/2025, 1:17:32.252 AM",GET /health,200,0.703879
1,"9/24/2025, 1:17:38.923 AM",POST /normal_v2/stream,200,5160.633589
2,"9/24/2025, 1:17:46.555 AM",GET /health,200,0.888682
3,"9/24/2025, 1:17:57.192 AM",GET /health,200,0.587366
4,"9/24/2025, 1:18:00.496 AM",POST /normal_v2/stream,200,4384.177237


In [46]:
# profile = ProfileReport(df, tsmode=True, sortby="timestamp", title="Time-Series EDA")
# profile.to_file("report_timeseries.html")

# replace all request url which pattern is /chat_with_data/[\w]{32} to /chat_with_data/session_id

# get all distinct request url
distinct_urls = df['request_line'].unique()
distinct_urls


array(['GET /health', 'POST /normal_v2/stream',
       'GET /chat_with_data/{session_id}',
       'POST /chat_with_data/file_list', 'POST /chat_with_data_v2/stream',
       'POST /web_search/stream', 'GET /', 'GET /mode/smes_modes',
       'GET /chat_with_data/file_types/', 'GET /multimodal/file_types/',
       'POST /deep_search/stream', 'POST /bing_mdi/stream',
       'POST /multimodal/file_list', 'GET /multimodal/{session_id}',
       'POST /multimodal_v2/stream', 'GET /robots933456.txt', 'GET /docs',
       'GET /openapi.json', 'POST /bot/api/messages',
       'POST /hr_v2/stream', 'GET /favicon.ico'], dtype=object)

In [51]:
# query all request having response_code != 200
error_requests = df[df['response_code'] != 200]
error_requests

,timestamp,request_line,response_code,response_time_ms
12645,"9/23/2025, 11:10:13.908 AM",GET /robots933456.txt,404,1.227195
12735,"9/23/2025, 11:14:44.553 AM",POST /bot/api/messages,415,0.356799
18105,"9/23/2025, 11:52:36.438 AM",POST /multimodal/file_list,400,143.156888
18515,"9/23/2025, 12:12:33.368 PM",GET /robots933456.txt,404,1.577016
18529,"9/23/2025, 12:12:57.159 PM",GET /robots933456.txt,404,1.400718
18554,"9/23/2025, 12:13:08.629 PM",GET /robots933456.txt,404,3.152849
20758,"9/23/2025, 10:13:49.916 AM",GET /robots933456.txt,404,1.264648
20759,"9/23/2025, 10:13:50.109 AM",GET /robots933456.txt,404,1.358847
20763,"9/23/2025, 10:14:02.912 AM",GET /robots933456.txt,404,1.176863
20771,"9/23/2025, 10:14:16.942 AM",GET /robots933456.txt,404,1.472447
